In [42]:
## import modules here
from pyspark import SparkContext, SparkConf
from time import time
import pickle


def createSC():
    conf = SparkConf()
    conf.setMaster("local[*]")
    conf.setAppName("C2LSH")
    sc = SparkContext(conf = conf)
    return sc

with open("toy/toy_hashed_data", "rb") as file:
    data = pickle.load(file)

with open("toy/toy_hashed_query", "rb") as file:
    query_hashes = pickle.load(file)

alpha_m = 10
beta_n = 10

sc = createSC()
data_hashes = sc.parallelize([(index, x) for index, x in enumerate(data)])


In [36]:
type(query_hashes)

list

In [44]:
def c2lsh(data_hashes, query_hashes, alpha_m, beta_n):

    offset, cand = 0, 0

#     query_hashes.cache()
    cand_set = set()
    length = len(query_hashes)
    
    while True:
        for t, v in data_hashes.groupByKey().collect():
            
            if count(v, query_hashes, offset, length) >= alpha_m:
                cand_set.add(t)
                
        if len(cand_set) < beta_n:
            offset += 1
        else:
            break
    
    rdd = sc.parallelize(cand_set)
    return rdd

res = c2lsh(data_hashes, query_hashes, alpha_m, beta_n).collect()

print(res)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [43]:
def count(data_hash, query_hashes, offset, length):
    count = 0

    for i in range(length):
        if abs(data_hash.data[0][i] - query_hashes[i]) <= offset:
            count += 1
    
    return count
    
    

In [ ]:
count = 0
for t in data_hashes.groupByKey().collect():
    for _ in t[1]:
        count+=1

count

In [22]:
dir(data_hashes)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_computeFractionForSampleSize',
 '_defaultReducePartitions',
 '_id',
 '_is_barrier',
 '_jrdd',
 '_jrdd_deserializer',
 '_memory_limit',
 '_pickled',
 '_reserialize',
 '_to_java_object_rdd',
 'aggregate',
 'aggregateByKey',
 'barrier',
 'cache',
 'cartesian',
 'checkpoint',
 'coalesce',
 'cogroup',
 'collect',
 'collectAsMap',
 'combineByKey',
 'context',
 'count',
 'countApprox',
 'countApproxDistinct',
 'countByKey',
 'countByValue',
 'ctx',
 'distinct',
 'filter',
 'first',
 'flatMap',
 'flatMapValues',
 'fold',
 'foldByKey',
 'foreach',
 'foreachPartition',
 'fullOuterJoin',
 'getCheckpo

In [40]:
k = set()
k.add(1)
rdd = sc.parallelize(k)
rdd
sc.stop()

In [48]:
z = data_hashes.map().count()
z

TypeError: map() missing 1 required positional argument: 'f'